In [1]:
import os
import numpy as np
import json
import string
from random import randint

import tensorflow as tf
import tensorflow.contrib.seq2seq as seq2seq

from tensorflow.python.ops.rnn_cell import GRUCell
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.python.ops.rnn_cell import DropoutWrapper, ResidualWrapper

from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.layers.core import Dense
from tensorflow.python.util import nest

from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.contrib.seq2seq.python.ops import beam_search_decoder

In [5]:
path = "../../data/MLDS_hw2_data/"
traindir = "training_data/feat/"

trainfiles = os.listdir(path+traindir)

traindata = {}
for i in range(0,len(trainfiles)):
    traindata[str.split(trainfiles[i],".")[0]+'.'+str.split(trainfiles[i],".")[1]] = np.load(path+traindir+trainfiles[i])
print(len(traindata))

trainjsonfile = open(path+"training_label.json","r")

trainjson = json.load(trainjsonfile)


words = []
maxlen = -1
for x in trainjson:
    for y in x['caption']:
        y = ''.join(c for c in y if c not in string.punctuation)
        ss = str.split(y," ")
        if len(ss)>maxlen:
            maxlen = len(ss)
            maxlenStr = y
        for z in ss:
            words.append(z.lower())
encodeWords = {}
counter = 4

for x in words:
    if x not in encodeWords:
        encodeWords[x] = counter
        counter = counter + 1 
encodeWords["<PAD>"] = 0      
encodeWords["<BOS>"] = 1
encodeWords["<EOS>"] = 2
encodeWords["<NAN>"] = 3
print(len(encodeWords))

decodeWords = {}
for key, value in encodeWords.items():
    decodeWords[value] = key
print(len(decodeWords))

np.save("encodeWords.npy",encodeWords)
np.save("decodeWords.npy",decodeWords)

max_seq_length = 21

def getStr(ints):
    sentence = ' '.join([decodeWords[int] for int in ints])
    sentence = sentence.replace('<BOS> ','').replace(' <EOS>', '')
    return sentence

def getMiniDataSets():
    x_data = np.zeros((1450,80,4096),dtype="float32")
    x_label = np.zeros((1450,max_seq_length),dtype="int32")
    x_label_train = np.zeros((1450,max_seq_length),dtype="int32")
    y_length = np.zeros((1450),dtype="int32")
    y_length_train = np.zeros((1450),dtype="int32")

    i = 0
    for x in trainjson:
        name = x["id"]
        temp = traindata[name]
        counter2 = 0
        
        random = randint(0, len(x["caption"])-1)
        
        y = x["caption"][random]

        x_data[i] = temp

        x_label_temp = []
        x_label_train_temp = []
        
        y = ''.join(c for c in y if c not in string.punctuation)
        temp = [encodeWords[x.lower()] for x in str.split(y," ")]
        
        if(len(temp)>20):
            temp = temp[:20]
        
        x_label_temp = temp + [encodeWords["<EOS>"]]
        x_label_train_temp = [encodeWords["<BOS>"]] + temp 
        
        y_length[i] = len(x_label_temp)
        y_length_train[i] = len(x_label_train_temp) 

        for xa in range(len(x_label_temp),max_seq_length):
            x_label_temp.append(encodeWords["<PAD>"])
        for xa in range(len(x_label_train_temp),max_seq_length):
            x_label_train_temp.append(encodeWords["<PAD>"])                       
            
        x_label_temp = np.reshape(x_label_temp,(max_seq_length))
        x_label_train_temp = np.reshape(x_label_train_temp,(max_seq_length))
        
        x_label[i] = x_label_temp
        x_label_train[i] = x_label_train_temp 
        i = i+1
        
    x_data = np.split(x_data,29)
    x_label = np.split(x_label,29)
    x_label_train = np.split(x_label_train,29)
    y_length = np.split(y_length,29)
    y_length_train = np.split(y_length_train,29)
    return x_data, x_label, x_label_train, y_length, y_length_train

1450
6061
6061


In [3]:
#tensorflow   

unit = 512
inputs = tf.placeholder(tf.float32,[None,80,4096]) 
labels = tf.placeholder(tf.int32,[None,max_seq_length])
labels_train = tf.placeholder(tf.int32,[None,max_seq_length])
length = tf.placeholder(tf.int32,[None])
length_train = tf.placeholder(tf.int32,[None])
batch_size = tf.shape(inputs)[0]
sequence_length = tf.fill([batch_size], max_seq_length)

def lstm_cell():
  return tf.contrib.rnn.BasicLSTMCell(unit)

encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(2)])

encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, inputs, dtype=tf.float32)
print(encoder_outputs.get_shape())

attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units=unit, memory=encoder_outputs)

decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(2)])
attention_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism)

initial_state = attention_cell.zero_state(dtype=tf.float32, batch_size=batch_size)
initial_state = initial_state.clone(cell_state=encoder_state) 

embedding = tf.Variable(tf.random_uniform([len(encodeWords), unit], -0.1, 0.1, dtype=tf.float32))
labels_embedded = tf.nn.embedding_lookup(embedding, labels_train)

output_projection_layer = Dense(len(encodeWords), use_bias=False)

#train
helper = tf.contrib.seq2seq.TrainingHelper(labels_embedded, sequence_length)
#helper = tf.contrib.seq2seq.ScheduledOutputTrainingHelper(labels_embedded, length_train,  0.5)
decoder = tf.contrib.seq2seq.BasicDecoder(attention_cell, helper, initial_state, output_layer=output_projection_layer)

decoder_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, maximum_iterations=max_seq_length)

outputs = decoder_outputs.rnn_output
sample = decoder_outputs.sample_id

masks = tf.cast(tf.sequence_mask(length, maxlen=max_seq_length),tf.float32);
loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=labels, weights=masks,average_across_timesteps=False,average_across_batch=True)
loss = tf.reduce_sum(loss)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
minimize = optimizer.minimize(loss)


(?, 80, 512)


In [4]:
trainCount = 0
totalLoss = 0
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for j in range(2000):
    x_data, x_label, x_label_train, y_length, y_length_train = getMiniDataSets()
    for i in range(29):
        trainCount = trainCount + 1
        
        _,l,predict = sess.run([minimize, loss, sample], 
                               feed_dict={
                                   inputs: x_data[i], 
                                   labels: x_label[i], 
                                   labels_train: x_label_train[i], 
                                   length: y_length[i],
                                   length_train: y_length_train[i]
                               })
        
        totalLoss += l
    ran = randint(0,49)
    log = "%d %f %s"%(j, totalLoss/trainCount, getStr(predict[ran]))
    print(log)
    if j%100==0:
        saver = tf.train.Saver()
        saver.save(sess, "model"+str(j)+".ckpt")

0 118.246693 <EOS>
1 110.747951 <EOS>
2 107.326029 a
3 106.898729 a
4 106.436371 a
5 105.375285 a is is is
6 103.819596 a is is is is
7 102.621860 a man is is
8 101.413931 a is is
9 100.258852 a is is is a
10 98.976463 a man is is is a
11 97.899083 a man is is is
12 97.040592 a man is is a
13 96.174852 a woman is a bowl a bowl bowl bowl
14 95.426711 a man is a is is a is
15 94.366641 a man is a knife knife
16 93.609277 a man is is a
17 93.101866 a man bowl a bowl
18 92.481835 a woman is is a bowl
19 92.068475 a man is a playing
20 91.656251 a man is is a
21 91.140784 a man is is a
22 90.479577 a man is playing a
23 89.886725 a man is playing a guitar a guitar
24 89.314320 a man is playing
25 88.992955 a man man is playing
26 88.515434 a woman is slicing a bowl
27 88.174445 a man man is the
28 87.678954 a man is riding a street
29 87.378088 a man is playing a other
30 86.888376 a man is a a street a street
31 86.519975 a woman is a bowl a a a a a a a a a a a a a a
32 86.243265 a man is 

163 63.524501 a man is a shower on on on on on
164 63.395859 a man is riding a horse on on on on on on on on on on on on on on
165 63.298851 a is slicing a apple into a sections into a a a apple into two sections into into
166 63.166229 a man is down a rifle a grass down in in in in in
167 63.059097 a woman is slicing a into into into into into into into into into into into into into into into
168 62.946459 a lady slicing a onion into into into into into into into into into into into into into into into
169 62.832153 a panda is is walking in log in in in in in in in in in
170 62.733040 a man is a picnic the a falls woman a is move him
171 62.616480 a is pouring bacon a into into into into into into into into into into
172 62.503062 a woman is riding a horse on on on on on on on on on on on on on on
173 62.388184 a woman is a egg in a bowl mixes the water into
174 62.280961 a are are running on a track on to on on on on on on on on on on
175 62.167015 a person is slicing a onion into in

263 53.671642 a boy throws throwing a into a camera ball into
264 53.588856 a people are smiling a a car over over over over over over over over over over over over
265 53.508841 a person animal is biting a a persons out out out out out out out out out out out
266 53.422454 a woman is applying eye makeup with to on on on on on on on on on on on on on
267 53.340164 a boy is a boy boy with with with with with with with with with with with with with with
268 53.258658 a animal is something piece of mango on on on on on on on on on on on on on
269 53.175733 three boys are dancing in in in in in in in in in in in in in in in
270 53.101103 a is rice is stirred stirred a pot pudding pudding pudding and and and
271 53.018337 two people are smiling a car are over are are are are are are are are are are are
272 52.933394 a man is throwing a in a boy is throwing in by standing by the
273 52.851552 a man is a hands on over a a over over in in over over over over over over
274 52.773511 a man is br

355 47.048346 a woman is riding a horse a rode on on on on on on on on on on
356 46.986241 a man is down a gun fired targets targets at at at in
357 46.922855 a man is shooting in a prone on in on on on in on on in in on on on
358 46.861026 a is an apple two a hands into into into into into into into into into into into
359 46.799329 a woman is slicing slicing a into into into into into into into into into into into into into
360 46.735110 a men are throwing in a into into into into into into into into into into into into into
361 46.675400 a person is slicing a a knife knife into into into into into into into into into into into
362 46.616736 a is put in a bowl discards into in and in in in in in in in in
363 46.553231 a panda is up tree on on on on on on on on on on on on on on
364 46.490621 a person is stirring rice in in in in in in in in in in in in in in in
365 46.425807 a girl is pulling a woman a hill for for for
366 46.363245 girls are cheering pom in the air in in into in in 

444 42.232209 two person is a pandas bottles a baby bottles out in bottles bottles bottles bottles bottles bottles bottles bottles
445 42.186228 a man is breaking boards boards his hand with with with with with with with with with
446 42.139753 a man is balancing on a rope rope in at in in in in in in in in in in
447 42.094599 a woman is twisting a twisting to a of do hair to to after do on after after after after
448 42.048081 a missile is going a ship in out out in in in in in in in in in in
449 41.999865 a woman is walking in down down down in in in in in in in in in in in in
450 41.949062 criss man is a women run on the beach come with the with then the three of them run
451 41.901286 a man man is riding a car smiling at a other at at in in in in at in
452 41.855439 a woman is is paddling a boat in in in in in in in in in in in in
453 41.807327 two people are riding a a car are are are in in in in in in in in in
454 41.762787 a elephant is shopping items at a pharmacy at at at at a

...

1611 19.679333 several runners are running on race track in toward well well well well well well well well well well well
1612 19.671349 three are dancing in in in in in in in in in in in in in in
1613 19.663010 a panda is a slide in into into into into into into into into into into into into into
1614 19.654650 a man snipes his rifle while laying on his stomach up position position into into into position into into into
1615 19.646527 a baby baby almost almost over on the couch on asleep over on over over over over
1616 19.638356 a lady is drinking milk bottles to two babies in bottles bottles bottles bottles bottles bottles bottles bottles bottles bottles
1617 19.630379 a woman is chopping an onion into in into into into like like like like like like like like like
1618 19.622406 a woman is an egg in in in into in in and into into into and into into into into into
1619 19.614747 a man is playing a on a stage at into and in into in in into in in in
1620 19.607151 a girl is rowing a bo

1694 19.038849 a lady cracked an egg into the mixer inside in into inside into into into into into into into
1695 19.031400 someone and rice is stired in in in in in in in in in in in in in in in
1696 19.023872 an elephant shopped for food in the drug store at its and and at
1697 19.016431 a woman is cracking an egg a bowl it open and open and into and and and and
1698 19.008834 a onion is cut in into into into into into into into into into into into into into
1699 19.001187 a man is milk in milk feeding bottles of two pandas in a cage bottles babies bottles
1700 18.993486 a woman woman is eyeliner on eye on on on with on on on on on on on
1701 18.985915 a man shoots shooting shooting at in in in in in in in in in in in in in in
1702 18.978310 a woman is dancing in in on in in in in in in in in in in in in
1703 18.970606 a boy is throwing tennis at another boy who is not catching but being hit by them but
1704 18.963028 a is slicing onions into into into into into into into into into i

1779 18.416665 a boy put his bark into his mouth about about about about in about about about about in
1780 18.410074 a horse and rider ran down a dirt rode toward down in in in in in in in in in
1781 18.403499 amanplaysaguitar man is playing guitar acoustic guitar in in in in in in in in in in in in
1782 18.396839 a woman is rowing a boat a shore over over over over over over over over over over
1783 18.390374 a panda is into a slide in his stomach and lands in the snow into into into into into
1784 18.383932 a puppy is playing with the grass from from from from from from from from from from
1785 18.377561 a lady chopped up an onion into into into into into into into into into into into into into
1786 18.371159 a woman is cracking an egg in the side of a bowl into water mixer and into into and into
1787 18.364812 a woman is slicing ginger vegetable of ginger thinly into into into into into into into into into into into
1788 18.358258 the two is girl are in a horse drawn carriage in in

1863 17.877386 a man fell down while running on the grass around and and and and and and and and and
1864 17.871060 a man is balancing on a rope the beach on at at at at at at at at at
1865 17.864797 a person is slicing ginger in into into into into into into into into into into and into into into
1866 17.858569 a bomb is a missile to blow up a boat through back back back back back back back back back
1867 17.852240 an elephant is a shopping basket in his trunk is putting a bottle of cough the and a box of
1868 17.845905 a submarine launched flying shot through through through through back back around around around back back back back
1869 17.839616 men are running down a track well well well well well well well well well well well well well well
1870 17.833665 rice person is stirring a pot of rice and pudding pudding pudding pudding pudding pudding pudding pudding
1871 17.827340 a man is flying in jet in the air to over as over over over over over over over over
1872 17.821347 three b

1946 17.349527 a man is talking to women on the beach on at on at at at at at at at at
1947 17.343197 three boys are dancing perform in to them to them to them them them them them them them
1948 17.336818 three horse horse is pulling on a pathway trotting a cart on which a woman and two girls are seated
1949 17.330523 a flies flies through the air over over over over over over over over over over over
1950 17.324148 a woman slicks in the rain herself back down in in back down down in back down back in
1951 17.317808 a submarine and is moving in the space through it into into into into into into into into into into
1952 17.311454 a man brought talking to women on at to at at at at at at at at at at at
1953 17.305138 a woman is scrambling an egg with a wire whisk it into into into into into into into into
1954 17.298826 a man shoots firing with a pistol at six in in in in in in in in in in in
1955 17.292573 a woman and rider ran down the dirt rode down down and down down down in down dow